In [2]:
import sys
sys.path.append("../src")

import pandas as pd
import numpy as np
from transformers import AutoTokenizer

from bert_model import BertModel
from embedding_model import EmbeddingModel
from collator import CustomCollator
from dataset import JuridiqueDataset
from preprocess import Preprocessing
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

from transformers import get_cosine_schedule_with_warmup

import random
import time

from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Import our datas

In [3]:
train_df = pd.read_csv("../datas/train_data.csv")

In [4]:
def remove_newlines(df):
    df = df.replace("\n", '', regex=True)
    return df
train_df = remove_newlines(train_df)

# Only keep the sex column

In [5]:
train = train_df.drop(["date_accident", "date_consolidation"], axis=1)
preprocess = Preprocessing(train)
train = preprocess.remove_stopwords()
train.head()

,ID,filename,texte,sexe
0,0,Agen_100515.txt,Le : 12/11/2019 Cour d’appel d’Agen chambre so...,homme
1,1,Agen_1100752.txt,Le : 12/11/2019 Cour d’appel d’Agen chambre ci...,homme
2,2,Agen_1613.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience p...,femme
3,3,Agen_2118.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience p...,femme
4,4,Agen_21229.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience p...,homme


In [6]:
train = train.drop(["ID", "filename"], axis=1)

In [7]:
train['text_id'] = np.arange(len(train))

# Let's train/test split our datas

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(train, test_size=0.2, random_state=42, stratify=train["sexe"])
df_train, df_test = df_train.copy(), df_test.copy()

# Let's call our created dataset

In [ ]:
model_name = "almanach/camembert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
args = {}
ds = JuridiqueDataset(train,tokenizer,args)
idx = random.choice(range(len(ds)))

# Now we use our DataLoader

In [ ]:
collator_function = CustomCollator(tokenizer)
my_dataset = JuridiqueDataset(train,tokenizer,args)

In [ ]:
data_loader = DataLoader(my_dataset,drop_last = False,num_workers=0,pin_memory=False,shuffle=False,
                              batch_size = 2,collate_fn = collator_function)

In [ ]:
for batch in tqdm(data_loader):
    break

# Training of the model

In [ ]:
def train_one_step(batch,model,criterion):
    """
    Complete this function which should return the loss generate on the bacth data
    """
    # convert bacth data to same device as model
    device  = torch.device(f"cuda:0" if torch.cuda.is_available() else "cpu")
    batch = batch_to_device(batch,device)
    # one step forward with the bacth
    pred = model(batch)

    # compute loss
    loss = criterion(pred.squeeze(),batch['sexe'].float().squeeze(-1))
    return loss

In [ ]:
def train_one_epoch(epoch_number,data_loader,model,criterion,optimzer,lr_scheduler):
    losses = []
    model.train()
    start_time = time.time()
    pbar = tqdm(data_loader)
    for batch in pbar:
        loss = train_one_step(batch,model,criterion)
        pbar.set_postfix({"loss":loss.item()})
        losses.append(loss.item())
        loss.backward()
        optimzer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    lr = scheduler.get_lr()[0]
    elapsed_time = time.time() - start_time
    loss_ = np.mean(losses)
    print(f"Epoch {epoch_number + 1} :  lr={lr:.6f} t={elapsed_time:.0f}s loss : {loss_:.5f}")
    return model

In [ ]:
def inference(valid_loader, model):
    predictions = []
    model.eval()

    device = next(model.parameters()).device

    with torch.no_grad():
        for batch in tqdm(valid_loader):
            batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
            pred = model(batch).sigmoid().squeeze()

            if pred.dim() == 0:
                pred = pred.unsqueeze(0)

            predictions.append(pred.detach().cpu().numpy())

    predictions = np.concatenate(predictions, axis=0)

    df_predict = pd.DataFrame({"sexe_pred": predictions.tolist()})
    return df_predict

In [ ]:
def batch_to_device(batch, device):
    """Moves only batch tensors to the specified device."""
    batch_dict = {}
    for key in batch:
        if isinstance(batch[key], torch.Tensor):
            batch_dict[key] = batch[key].to(device)
        else:
            batch_dict[key] = batch[key]
    return batch_dict


In [ ]:
# Define your model
device = torch.device(f"cuda:0" if torch.cuda.is_available() else "cpu")
net = BertModel(model_name=model_name,num_labels=1)
net.to(device)

# Define an optimzer
optimizer = optim.AdamW(net.parameters(),lr = 4e-6 )

# Define a scheduller for your model training

BATCH_SIZE = 8
EPOCHS = 10
warmup_steps = 0.04 * (len(train)//BATCH_SIZE)
training_steps = EPOCHS * (len(train)// (BATCH_SIZE))

scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, training_steps)

In [ ]:
criterion = nn.BCEWithLogitsLoss().to(device)
for epoch_num in range(EPOCHS):
    net = train_one_epoch(epoch_num,data_loader,net,criterion,optimizer,scheduler)

In [ ]:
collator_function = CustomCollator(tokenizer)
valid_dataset = JuridiqueDataset(df_test, tokenizer,args)
valid_loader = DataLoader(valid_dataset,drop_last = False,num_workers=0,pin_memory=False,shuffle=False,
                              batch_size = 8, collate_fn = collator_function)

In [ ]:
pred_df = inference(valid_loader,net)

# Let's predict the sex of the victim!

In [ ]:
def get_embeddings(model, df):
    collator_function = CustomCollator(tokenizer)
    valid_dataset = JuridiqueDataset(df, tokenizer, args)
    valid_loader = DataLoader(valid_dataset, drop_last=False, num_workers=0, pin_memory=False, shuffle=False,
                              batch_size=2, collate_fn=collator_function)
    
    embed_predictions = []
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    model.eval()
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            batch = batch_to_device(batch, device)
            pred = model(batch)

            if pred.dim() == 1:
                pred = pred.unsqueeze(0)
            pred = pred.detach().cpu().numpy()
            embed_predictions.append(pred)
    
    embeddings = np.concatenate(embed_predictions, axis=0)
    df_predict = pd.DataFrame(embeddings)
    
    return df_predict

In [ ]:
device = torch.device(f"cuda:0" if torch.cuda.is_available() else "cpu")
net = EmbeddingModel(model_name=model_name)
net.to(device)

In [ ]:
df_embed_train = get_embeddings(net,df_train)
df_embed_test = get_embeddings(net,df_test)

In [ ]:
X = df_embed_train

df_train["sexe"] = df_train["sexe"].replace({"homme":1,
                      "femme":0,
                      "n.c.":-1})
df_test["sexe"] = df_test["sexe"].replace({"homme":1,
                      "femme":0,
                      "n.c.":-1})
y = df_train['sexe']

clf = LogisticRegression(multi_class='multinomial', random_state=26).fit(X, y)

preds = clf.predict(df_embed_test)

y_test = df_test["sexe"]


# Now, we compute our metrics

In [14]:
acc = metrics.accuracy_score(y_test, preds)
f1 = metrics.f1_score(y_test, preds, average='macro')

print("accuracy of the embeddings model:", 100*round(acc,2),"%")
print("f1 of the embeddings model::", 100*round(f1,2),"%")

accuracy of the embeddings model: 72.0 %
f1 of the embeddings model:: 49.0 %
